In [9]:
from pyomo.environ import *
from pyomo.opt import SolverFactory 
import pandas as pd

# Modelo concreto
model = ConcreteModel()

# Lectura de datos desde Excel
L_cv_data = pd.read_excel(r'/Users/allis/Downloads/PYC/DATA.xlsx', sheet_name="L_cv", index_col=[0, 1])
CS_cv_data = pd.read_excel(r'/Users/allis/Downloads/PYC/DATA.xlsx', sheet_name="CS_cv", index_col=[0, 1])
STC_cvp_data = pd.read_excel(r'/Users/allis/Downloads/PYC/DATA.xlsx', sheet_name="STC_cvp", index_col=[0, 1, 2])
U_p_data = pd.read_excel(r'/Users/allis/Downloads/PYC/DATA.xlsx', sheet_name="U_p", index_col=[0])
Lab_p_data = pd.read_excel(r'/Users/allis/Downloads/PYC/DATA.xlsx', sheet_name="Lab_p", index_col=[0])
MC_p_data = pd.read_excel(r'/Users/allis/Downloads/PYC/DATA.xlsx', sheet_name="MC_p", index_col=[0])
PTC_pd_data = pd.read_excel(r'/Users/allis/Downloads/PYC/DATA.xlsx', sheet_name="PTC_pd", index_col=[0, 1])
SP_d_data = pd.read_excel(r'/Users/allis/Downloads/PYC/DATA.xlsx', sheet_name="SP_d", index_col=[0])
D_d_data = pd.read_excel(r'/Users/allis/Downloads/PYC/DATA.xlsx', sheet_name="D_d", index_col=[0])

# Definición de conjuntos
components = {1, 2, 3}  # 3 componentes
vendors = {1, 2, 3}     # 3 proveedores
plants = {1, 2}         # 2 plantas
distribution_centers = {1, 2, 3, 4, 5, 6}  # 6 centros de distribución

# Inicialización de conjuntos en el modelo
model.C = Set(initialize=components)
model.V = Set(initialize=vendors)
model.P = Set(initialize=plants)
model.D = Set(initialize=distribution_centers)

# Variables de decisión
model.X = Var(model.C, model.V, model.P, within=NonNegativeReals, doc="Cantidad de componentes enviados del proveedor a la planta")
model.Y = Var(model.P, model.D, within=NonNegativeReals, doc="Cantidad de productos enviados de la planta al centro de distribución")

# Parámetros
model.L_cv = Param(model.C, model.V, initialize=L_cv_data['L_cv'].to_dict(), default=0) 
model.CS_cv = Param(model.C, model.V, initialize=CS_cv_data['CS_cv'].to_dict(), default=0) 
model.STC_cvp = Param(model.C, model.V, model.P, initialize=STC_cvp_data['STC_cvp'].to_dict(), default=0) 
model.U_p = Param(model.P, initialize=U_p_data['U_p'].to_dict(), default=0) 
model.Lab_p = Param(model.P, initialize=Lab_p_data['Lab_p'].to_dict(), default=0) 
model.MC_p = Param(model.P, initialize=MC_p_data['MC_p'].to_dict(), default=0)  
model.PTC_pd = Param(model.P, model.D, initialize=PTC_pd_data['PTC_pd'].to_dict(), default=0) 
model.SP_d = Param(model.D, initialize=SP_d_data['SP_d'].to_dict(), default=0)
model.D_d = Param(model.D, initialize=D_d_data['D_d'].to_dict(), default=0)

# Función objetivo: minimizar el costo operativo total de la cadena de suministro (TSCC)
def objective_rule(model):
    # (a) Costo total de materiales del proveedor (TSMC)
    TSMC = sum(model.CS_cv[c, v] * model.X[c, v, p] for c in model.C for v in model.V for p in model.P)
    
    # (b) Costo total de fabricación (TMC)
    TMC = sum(model.MC_p[p] * sum(model.Y[p, d] for d in model.D) for p in model.P) 

    # (c) Costo total de transporte (TTC)
    TTC = sum(model.X[c, v, p] * model.STC_cvp[c, v, p] for c in model.C for v in model.V for p in model.P) + \
          sum(model.Y[p, d] * model.PTC_pd[p, d] for p in model.P for d in model.D)
    
    # Costo total del sistema (TSCC)
    return TSMC + TMC + TTC

model.obj = Objective(rule=objective_rule, sense=minimize)

# Restricciones
# Restricción 1: capacidad del proveedor
def capacity_constraint_v(model, c, v):
    return sum(model.X[c, v, p] for p in model.P) <= model.L_cv[c, v]
model.capacity_constraint_v = Constraint(model.C, model.V, rule=capacity_constraint_v)

# Restricción 2: capacidad de la planta
def capacity_constraint_p(model, p):
    return sum(model.Y[p, d] for d in model.D) <= model.U_p[p]
model.capacity_constraint_p = Constraint(model.P, rule=capacity_constraint_p)

# Restricción 3: demanda en los centros de distribución
def demand_constraint_d(model, d):
    return sum(model.Y[p, d] for p in model.P) >= model.D_d[d]
model.demand_constraint_d = Constraint(model.D, rule=demand_constraint_d)

# Restricción de  flujo de inventario en las plantas 
def flow_constraint(model, c, p):
    return sum(model.X[c, v, p] for v in model.V) - sum(model.Y[p, d] for d in model.D) >= 0
model.flow_constraint = Constraint(model.C, model.P, rule=balance_constraint)

# Resolver el modelo
opt = SolverFactory('glpk')
results = opt.solve(model)

# Mostrar resultados más claros
print("\n--- Resultados ---")
print(f"Costo total del sistema (TSCC):$ {model.obj()}")

# Mostrar las decisiones de envío del proveedor a la planta
print("\nCantidad enviada de proveedor a planta (X):")
for c in model.C:
    for v in model.V:
        for p in model.P:
            if model.X[c, v, p]() > 0:
                print(f"Componente {c}, Proveedor {v}, Planta {p}: {model.X[c, v, p]()}")

# Mostrar las decisiones de envío de planta a centro de distribución
print("\nCantidad enviada de planta a centro de distribución (Y):")
for p in model.P:
    for d in model.D:
        if model.Y[p, d]() > 0:
            print(f"Planta {p}, Centro de Distribución {d}: {model.Y[p, d]()}")
print("\n") 

(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo
(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo
(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo
(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo

--- Resultados ---
Costo total del sistema (TSCC):$ 1106053.0

Cantidad enviada de proveedor a planta (X):
Componente 1, Proveedor 1, Planta 1: 100.0
Componente 1, Proveedor 1, Planta 2: 72.0
Componente 1, Proveedor 3, Planta 1: 300.0
Componente 2, Proveedor 1, Planta 1: 78.0
Componente 2, Proveedor 1, Planta 2: 72.0
Componente 2, Proveedor 2, Planta 1: 322.0
Componente 3, Proveedor 1, Planta 1: 72.0
Componente 3, Proveedor 2, Planta 1: 150.0
Componente 3, Proveedor 3, Planta 1: 178.0
Componente 3, Proveedor 3, Planta 2: 72.0

Cantidad enviada de planta a centro de distribución (Y):
Planta 1, Centro de Distribución 1: 83.0
Planta 1, Centro de Distribución 2: 98.0
Planta 1, C